## 10.3 Fixed Effects Regression

- Consider the panel regression model $Y_{it} = \beta_0 + \beta_1 X_{it} + \beta_2 Z_i +  u_{it}$ where the $Z_i$ are unobserved time-invariant heterogeneities across the entities $i=1,…,n$.
- We aim to estimate $β_1$, the effect on $Y_i$ of a change in $X_i$ holding constant $Z_i$.
- Letting $α_i=β_0+β_2Z_i$ we obtain the model $Y_{it} = \alpha_i + \beta_1 X_{it} + u_{it}$.
- The $α_i$ are entity-specific intercepts that capture heterogeneities across entities. 
- Having individual specific intercepts $α_i$, $i=1,…,n$, where each of these can be understood as the fixed effect of entity $i$, this model is called the fixed effects model.
- The variation in the $α_i$, $i=1,…,n$ comes from the $Z_i$.
- The fixed effects model could also be rewritten as a regression model containing $n−1$ dummy regressors and a constant:

\begin{align}
Y_{it} = \beta_0 + \beta_1 X_{it} + \gamma_2 D2_i + \gamma_3 D3_i + \cdots + \gamma_n Dn_i + u_{it}
\end{align}

### Estimation and Inference

- Software packages use a so-called “entity-demeaned” OLS algorithm which is computationally more efficient than estimating regression models with $k+n$ regressors.
- Taking averages on both sides of $Y_{it} = \alpha_i + \beta_1 X_{it} + u_{it}$ we obtain 

\begin{align*}
\frac{1}{n} \sum_{i=1}^n Y_{it} =& \, \beta_1 \frac{1}{n} \sum_{i=1}^n X_{it} + \frac{1}{n} \sum_{i=1}^n a_i + \frac{1}{n} \sum_{i=1}^n u_{it} \\
\overline{Y} =& \, \beta_1 \overline{X}_i + \alpha_i + \overline{u}_i. 
\end{align*}

- Subtraction from $Y_{it} = \alpha_i + \beta_1 X_{it} + u_{it}$ yields 

\begin{align}
\begin{split}
Y_{it} - \overline{Y}_i =& \, \beta_1(X_{it}-\overline{X}_i) + (u_{it} - \overline{u}_i) \\
\overset{\sim}{Y}_{it} =& \, \beta_1 \overset{\sim}{X}_{it} + \overset{\sim}{u}_{it}. 
\end{split}
\end{align}

- We conclude that there are two ways of estimating $β_1$ in the fixed effects regression:
 1. OLS of the dummy regression model
 2. OLS using the entity demeaned data
 
 
- Provided the fixed effects regression assumptions stated in Key Concept 10.3 hold, the sampling distribution of the OLS estimator in the fixed effects regression model is normal in large samples.
- The variance of the estimates can be estimated and we can compute standard errors,  t-statistics and confidence intervals for coefficients. 

### Application to Traffic Deaths

- The simple fixed effects model for estimation of the relation between traffic fatality rates and the beer taxes is 

\begin{align}
FatalityRate_{it} = \beta_1 BeerTax_{it} + StateFixedEffects + u_{it}
\end{align}

- We can include fixed effects in our model by simply adding the fixed effect variables using fe().

In [2]:
using CSV
using DataFrames
using Query
using Plots
using FixedEffects
using FixedEffectModels
using CategoricalArrays
using LinearAlgebra

fatalities = CSV.read("/mnt/juliabox/Econometrics With Julia/Datasets/fatalities.csv")

fatalities.fatality_rate = fatalities.fatal ./ fatalities.pop * 10000
fatalities.state = categorical(fatalities.state)

fatalities_model = reg(
                        fatalities,
                        @model(fatality_rate ~ beertax + fe(state)),
)

println(fatalities_model)

x = fatalities.beertax
y = fatalities.fatality_rate

p1 = plot( #assign a plot object to the variable p1 using the following attributes
    x, #x series
    y, #y series
    st = :scatter, #series type
    title = "Traffic Fatality Rates and Beer Taxes from 1982-1988", #plot title
    label = "Observation", #legend labels
    xlabel = "Beer tax (in 1988 dollars)", #x axis label
    ylabel = "Fatality rate (fatalities per 10000)", #y axis label
    ylims = (0, 5), #y axis limits
    yticks = 0:0.5:5, #y axis tick range
    ms = 4, #marker size
    mc = :blue #marker color
)

y_prediction_diff(x) = dot(coef(fatalities_model), [1, x])

x = [minimum(fatalities.beertax), maximum(fatalities.beertax)]
y = y_prediction_diff.(x)

plot!(
    p1,
    x,
    y,
    st = :line,
    label = "OLS Difference"
)

display(p1)

MethodError: MethodError: no method matching length(::Expr)
Closest candidates are:
  length(!Matched::Core.SimpleVector) at essentials.jl:582
  length(!Matched::Base.MethodList) at reflection.jl:732
  length(!Matched::Core.MethodTable) at reflection.jl:806
  ...

- The estimated coefficient is −0.6559

\begin{align}
\widehat{FatalityRate} = -\underset{(0.29)}{0.66} \times BeerTax + StateFixedEffects
\end{align}
 